In [ ]:
suppressMessages({
    library(tidyverse)
    library(data.table)

})


save_plot <- function(p, fn, w, h){
    for(ext in c(".pdf", ".png")){
        ggsave(filename=paste0(fn,ext), plot=p, width=w, height=h)
    }
}

dir.create('../_m/')
setwd('../_m/')

In [ ]:
#file_list

In [ ]:
# create a list of all the filenames in the folder that end with ".csv"
file_list <- Sys.glob('../../custom_universe*/*/_m/*_gene_set_results.gsa.out')
#file_list
file_list <- file_list[file.info(file_list)$size > 0] %>% # remove empty files (no magma enrichment)
                discard(~str_detect(.x, '/custom_universe/blood_traits/_m/Cv*|/custom_universe/psych_general_traits/_m/Cv*')) # remove DE direction of effect from specific analysis for brain region x brain region 
file_list %>% head()

study_name <- gsub('.*.txt_|_gene_set.*','',file_list)
study_name %>% head()


brain_region <- gsub('.*_m/|_deg.genes.txt.*','',file_list) %>%
                gsub('_.*','',.)
brain_region <- lapply(brain_region, toupper) %>% as.character()

brain_region[brain_region == "CVD"] <- "CAUDATExDLPFC"
brain_region[brain_region == "CVH"] <- "CAUDATExHIPPOCAMPUS"

table(brain_region)
head(brain_region)




comparison <- gsub('.*_m/|_deg.genes.txt.*|','',file_list) %>%
                gsub('.*_','',.)

comparison[comparison == "betweensex"] <- "sex_by_sz"
comparison[comparison == "betweensex"] <- "sex_by_sz"



comparison %>% table()



#_betweensex_deg_genes: sex_by_sz

#XvD: region_by_sex

#maleVfemale: maleVfemale

In [ ]:
df <- data.frame()

for (i in seq_along(file_list)) {
    dftmp <- fread(file_list[i], skip = 4) %>%
        set_names(c('VARIABLE', 'TYPE', 'NGENES', 'BETA', 'BETA_STD', 'SE', 'P')) %>%
        mutate(GWAS_set = study_name[i],
               Status = ifelse(grepl('down', VARIABLE), 'Downregulated', 'Upregulated'),
               Brain_Region = brain_region[i],
               DEComparison = comparison[i]
        ) %>%
        mutate(GWAS_set = ifelse(GWAS_set == 'SCZ', 'SCHIZOPHRENIA', GWAS_set)) %>%
        group_by(Brain_Region, DEComparison) %>%
        mutate(fdr_bh = p.adjust(P, method = "BH"),
               fdr_bonferroni = p.adjust(P, method = "bonferroni")) %>%
        select(VARIABLE,TYPE, NGENES, BETA, BETA_STD, SE, P, GWAS_set, fdr_bh, fdr_bonferroni, Status ,Brain_Region, DEComparison)
    df <- rbind(dftmp, df) %>% arrange(GWAS_set)
}

df <- df %>% 
        mutate(Status = ifelse(DEComparison == 'all', 'Both', Status),
               DEComparison = ifelse(DEComparison == 'all','region_by_sex', DEComparison))


head(df)
fwrite(df, 'magma_enrichment_results.tsv', quote = F, sep = '\t', row.names = F)


In [ ]:
subset(df , P < 0.05)

In [ ]:
df %>% arrange(desc(abs(BETA))) %>% head(20)

In [ ]:
sessionInfo()